In [1]:
import database
from invoicerecord import MedicationRecord
import datetime
#database.db.create_all()

In [7]:
test = MedicationRecord(
    id=123456,
    name="test medication_blah",
    common_name="test",
    transactions=[MedicationRecord.transaction(
        date=datetime.datetime(2017,12,4),
        price="3.78",
        qty=10)
        ],
    dosage="once daily",
    admin="PO",
    category="debug",
    prescribable=False,
    aliases = None
    )

In [4]:
test.transactions[0].price

'3.78'

In [8]:
testdb = database.save_persistent_record(test)

AttributeError: 'list' object has no attribute '_sa_instance_state'

In [6]:
testdb

<PersistentMedication 3>